# Import BRAILS Importer

In [1]:
from brails.utils import Importer

# Define Location Specific Parameters

In [2]:
LOCATION_NAME = 'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 0

# Create and Importer object to Pull In Required BRAILS Modules

In [3]:
importer = Importer()

# Define the Region Object for the Region of Interest

In [4]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

# Get Raw NSI Data for the Defined Region

In [5]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States


INFO:root:
Getting National Structure Inventory (NSI) building data for the entered location...



Found a total of 3558 building points in NSI that are within the entered region of interest


# Get FEMA USA Footprints Data for the Defined Region

In [7]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Meshing the defined area...


Obtaining the number of elements in each cell: 100%|█| 10/10 [00:01<00:00,  8.24



Meshing complete. Split Fort Myers Beach into 10 cells


Obtaining the building attributes for each cell: 100%|█| 10/10 [00:01<00:00,  8.
100%|████████████████████████████████████████| 10/10 [00:00<00:00, 30393.51it/s]



Found a total of 3102 building footprints in Fort Myers Beach


# Create a Baseline Inventory by Merging NSI Raw Data and USA Structures Footprints

In [8]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True, add_features=[])


Getting National Structure Inventory (NSI) building data for the entered location...
Found a total of 2796 building points in NSI that match the footprint data.


# Fill Gaps in the Baseline Inventory by Imputing Missing Values

In [9]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

Existing worlds: 1
New worlds per existing world: 0
world # 0
Features with no reference data cannot be imputed. Removing them from the imputation target: buildingheight
Missing percentages among 3102 assets
fparea: 9.86%
numstories: 9.86%
erabuilt: 9.86%
repaircost: 9.86%
constype: 9.86%
occupancy: 9.86%
found_ht: 9.86%
ground_elv: 9.86%
FloodZone: 9.86%
splitlevel: 9.86%
basement: 28.11%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.01 mins


# Predict Roof Type Information from Aerial Imagery

In [ ]:
google_satellite_class = importer.get_class('GoogleSatellite')
google_satellite = google_satellite_class()
images_aerial = google_satellite.get_images(imputed_inventory,
                                            'tmp/satellite/')

roof_shape_classifier_class = importer.get_class('RoofShapeClassifier')
roof_shape_classifier = roof_shape_classifier_class()
predictions = roof_shape_classifier.predict(images_aerial)

Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/inventory_creation/tmp/satellite


Obtaining satellite imagery:   0%|             | 7/3102 [00:00<01:42, 30.24it/s]

Saved image to tmp/satellite/imsat_26.40610762-81.88609521.jpg
Saved image to tmp/satellite/imsat_26.40527095-81.88895908.jpg
Saved image to tmp/satellite/imsat_26.40499353-81.8936453.jpg
Saved image to tmp/satellite/imsat_26.40619151-81.8871131.jpg
Saved image to tmp/satellite/imsat_26.40588159-81.88805157.jpg
Saved image to tmp/satellite/imsat_26.40624176-81.89602984.jpg
Saved image to tmp/satellite/imsat_26.40417141-81.89001474.jpg
Saved image to tmp/satellite/imsat_26.40575555-81.89314358.jpg
Saved image to tmp/satellite/imsat_26.40632404-81.88521955.jpg
Saved image to tmp/satellite/imsat_26.40503011-81.88341606.jpg
Saved image to tmp/satellite/imsat_26.4069747-81.89170575.jpg
Saved image to tmp/satellite/imsat_26.40601081-81.89246931.jpg
Saved image to tmp/satellite/imsat_26.40415864-81.88858949.jpg
Saved image to tmp/satellite/imsat_26.40544011-81.88528761.jpg
Saved image to tmp/satellite/imsat_26.40480433-81.88679908.jpg
Saved image to tmp/satellite/imsat_26.406855-81.8919772.jp

Obtaining satellite imagery:   1%|            | 26/3102 [00:00<00:53, 57.72it/s]

Saved image to tmp/satellite/imsat_26.40507281-81.89231712.jpg
Saved image to tmp/satellite/imsat_26.40671348-81.89244082.jpg
Saved image to tmp/satellite/imsat_26.40654066-81.8840436.jpg
Saved image to tmp/satellite/imsat_26.40520751-81.89116456.jpg
Saved image to tmp/satellite/imsat_26.40474153-81.8877192.jpg
Saved image to tmp/satellite/imsat_26.40559935-81.88560429.jpg
Saved image to tmp/satellite/imsat_26.404761-81.89093594.jpg
Saved image to tmp/satellite/imsat_26.40470189-81.89038664.jpg
Saved image to tmp/satellite/imsat_26.40531515-81.88695107.jpg
Saved image to tmp/satellite/imsat_26.40444172-81.89210926.jpg
Saved image to tmp/satellite/imsat_26.40660402-81.8846271.jpg
Saved image to tmp/satellite/imsat_26.40476905-81.88702854.jpg
Saved image to tmp/satellite/imsat_26.40503309-81.8896602.jpg
Saved image to tmp/satellite/imsat_26.40549485-81.88858255.jpg
Saved image to tmp/satellite/imsat_26.40599199-81.89439253.jpg


...

Obtaining satellite imagery: 100%|█████████▉| 3093/3102 [00:50<00:00, 79.62it/s]

Saved image to tmp/satellite/imsat_26.45832398-81.96024956.jpg
Saved image to tmp/satellite/imsat_26.4571212-81.95919391.jpg
Saved image to tmp/satellite/imsat_26.45701162-81.9589866.jpg
Saved image to tmp/satellite/imsat_26.45614715-81.95929701.jpg
Saved image to tmp/satellite/imsat_26.45778606-81.96108841.jpg
Saved image to tmp/satellite/imsat_26.45881906-81.96296403.jpg
Saved image to tmp/satellite/imsat_26.45670246-81.95926985.jpg
Saved image to tmp/satellite/imsat_26.45757617-81.95996472.jpg
Saved image to tmp/satellite/imsat_26.46319169-81.96613042.jpg
Saved image to tmp/satellite/imsat_26.45901935-81.9623288.jpg
Saved image to tmp/satellite/imsat_26.46344728-81.96616412.jpg
Saved image to tmp/satellite/imsat_26.46203451-81.96550577.jpg
Saved image to tmp/satellite/imsat_26.45553478-81.95762596.jpg
Saved image to tmp/satellite/imsat_26.4586471-81.96274542.jpg
Saved image to tmp/satellite/imsat_26.45727378-81.96075606.jpg
Saved image to tmp/satellite/imsat_26.46098093-81.96522498.

Obtaining satellite imagery: 100%|██████████| 3102/3102 [00:51<00:00, 59.92it/s]

Saved image to tmp/satellite/imsat_26.45740921-81.95778684.jpg



Default roof classifier model in tmp/models/roofTypeClassifier_v1.pth loaded


# Getting Design Wind Speed, Flood Zone and Roof Type Information ,..., in Created Inventory

In [ ]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({'DesignWindSpeed': 130,
                      'FloodZone': 'AE',
                      'RoofShape': predictions[key],
                      'AvgJanTemp':'Above',
                      'NumberOfUnits':1,
                      'LandCover': 'Open'})

# Run Rulesets to Infer R2D-Required Data for Hazus Damage and Loss model

In [ ]:
imputed_inventory.change_feature_names({'fparea': 'PlanArea',
                                        'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingType',
                                        'numstories': 'NumberOfStories',
                                        'occupancy': 'OccupancyClass'})
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer = hurricaneInferer(
    input_inventory=imputed_inventory, clean_features=True)
hazus_inferred_inventory = inferer.infer()

# Validate the inventory and make correction if needed

In [ ]:
invalid_id, error_record = inferer.validate(hazus_inferred_inventory)

In [ ]:
inventory_corrrected = inferer.correct(hazus_inferred_inventory, invalid_id=invalid_id, weights={'StructureType':2})

# Write the Created Inventory in a GeoJSON File

In [ ]:
_ = inventory_corrrected.write_to_geojson(
    output_file=INVENTORY_OUTPUT)